# Ayudantía 5
## Grafos

_Form Feedback:_ https://goo.gl/forms/aFpTe9LGx9ObcMBu1

**Autores: Pablo Polea e Ian Basly**

## Problema:

 <center><img src="images/AY-1.png" alt="image2" width="90%" weight="90%"></center>

## Clase Nodo

In [ ]:
class Nodo:
    def __init__(self, codigo):
        self.codigo = codigo
        self.conexiones = []
    
    def agregar_nodo(self, nodo):
        self.conexiones.append(nodo)
    
    def quitar_nodo(self, nodo):
        self.conexiones.remove(nodo)
    
    def __str__(self):
        return f"[{self.codigo}]"

## Clase Grafo

In [ ]:
class Grafo:
    def __init__(self):
        self.nodos = dict()
    
    def agregar_nodo(self, nodo):
        self.nodos[nodo.codigo] = nodo
    
    def conectar(self, nodo_A, nodo_B):
        nodo_a = self.nodos[nodo_A]
        nodo_b = self.nodos[nodo_B]
        
        nodo_a.agregar_nodo(nodo_b)
        
        #print(f"{nodo_b} se ha conectado a {nodo_a}")
    
    def remover_conexion(self, nodo_A, nodo_B):
        nodo_a = self.nodos[nodo_A.codigo]
        nodo_b = self.nodos[nodo_A.codigo]
        
        nodo_a.quitar_nodo(nodo_b)
        
        #print(f"{nodo_b} se ha desconectado de {nodo_a}")
    
    def hay_ruta(self, nodo_A, nodo_B):
        nodo = self.nodos[nodo_A]
        
        visitados = set()
        faltantes = [nodo]
        
        while len(faltantes) > 0:
            nodo = faltantes.pop()
            if nodo.codigo == nodo_B:
                return True
            if nodo not in visitados:
                visitados.add(nodo)
                for hijo in nodo.conexiones:
                    faltantes.append(hijo)
        return False
            
    
    def ruta(self, nodo_A, nodo_B, visitados = set()):
        nodo = self.nodos[nodo_A]
        
        ruta = [nodo] #Como la ruta debe contener al nodo de inicio, se define inicialmente así
        
        if nodo.codigo == nodo_B: 
            return ruta #En caso de que el nodo en el que estamos sea nuestro destino, retornamos la ruta (caso base)
            # Al aplicar return se termina la función
        
        visitados.add(nodo) 
        
        for hijo in nodo.conexiones:
            if hijo not in visitados:
                r_hijo = self.ruta(hijo.codigo, nodo_B, visitados) #Ruta siempre retorna una lista
                if r_hijo: #Si la lista contiene algo, entonces de alguna forma ya se llego al destino
                    ruta.extend(r_hijo) #Agregamos a la ruta local, lo que la ruta de el hijo contenga
                    break
        return ruta if len(ruta) > 1 else [] 
    #Si nuetra ruta es de largo 1, significa que solo es el nodo actual 
    #y por lo tanto, los hijos tampoco pudieron llegar al destino.
    # Se retorna una lista vacía indicando que no hay ruta.
        

Hay que entender que la lista `ruta` solo se retorna con datos cuando llega al caso base (destino), entonces solo ahí `r_hijo` recive una lista no vacía, entrando al `if` y agregándose a la ruta local del padre y así sucesivamente generando una especie de reacción en cadena. 
De lo contrario solo se retornará una lista vacía que indica que desde ese nodo y sus hijos no se llegó de ninguna forma al nodo de destino.

## Archivos

In [ ]:
def leer_archivos(archivo_nodos, archivo_conexiones):
    grafo = Grafo()
    
    with open(archivo_nodos, "r") as nodos:
        for linea in nodos:
            nodo = Nodo(linea.strip())
            grafo.agregar_nodo(nodo)
        
    with open(archivo_conexiones, "r") as conexiones:
        for linea in conexiones:
            nodo_a, nodo_b = linea.strip().split(",")
            grafo.conectar(nodo_a, nodo_b)
        
    return grafo

## A jugar!!

In [ ]:
grafo = leer_archivos("nodos.txt", "conexiones.txt")

In [ ]:
grafo.hay_ruta("P001","P502")
    

In [ ]:
ruta = grafo.ruta("P001","P502")
print(" -> ".join(str(n) for n in ruta))

Para entontrar la ruta mas corta, primero se define una funcion auxiliar `ruta_rec` que nos entregue todas las rutas posibles desde A hasta B.

### Propuesto 😏: Ruta más corta

Para entontrar la ruta mas corta, primero se define una funcion auxiliar `ruta_rec` que nos entregue todas las rutas posibles desde A hasta B.

In [ ]:
def ruta_rec(grafo, nodo_A, nodo_B, visitados = set()):
    nodo = grafo.nodos[nodo_A]
        
    rutas = [] # Aquí rutas, es una lista que contendrá a todas las posibles rutas, donde cada una será otra lista.
        
    if nodo.codigo == nodo_B:
        
        rutas.append([nodo]) # Caso base, si el nodo actual es el destino agregamos la ruta y retornamos la lista.
        
        return rutas
    
    visitados.add(nodo)
    
    for hijo in nodo.conexiones:
        if hijo not in visitados:
            
            r_hijo = ruta_rec(grafo, hijo.codigo, nodo_B, visitados.copy()) # VER NOTA ABAJO
            # Al igual que antes rutas_rec siempre retorna una lista
            
            for ruta in r_hijo: #Ya que esta vez lo retornado es una lista de listas, recorremos cada ruta
                ruta.insert(0, nodo) #Agregamos el nodo actual a la ruta
                
                rutas.append(ruta) #agregamos esta ruta a las rutas locales
                    
    return rutas

In [ ]:
rutas = ruta_rec(grafo, "P001", "P502")
for ruta in rutas:
    print(" -> ".join(str(n) for n in ruta))

**NOTA:**

Esta vez se agrega una copia de visitados, ya que queremos todas las rutas desde este punto y eso puede significar que por cada hijo podriamos pasar por un mismo nodo, pero no queremos pasar por uno que nuestos padres hayan visitados para no formar ciclos.

De esta forma se da independencia de visitados futuros a cada hijo, manteniendo los mismos visitados hasta ese punto.

In [ ]:
def ruta_mas_corta(grafo, nodo_A, nodo_B):
    rutas = ruta_rec(grafo, nodo_A, nodo_B)
    
    rutas.sort(key = lambda ruta: len(ruta)) # Solo se ordenan las rutas por largo.
    
    return rutas[0]

Luego `ruta_mas_corta` ordena estas rutas y encuentra la de menor largo. Puede ocurrir que hayan más de una "ruta más corta" pero tomaremos la primera.

In [ ]:
ruta = ruta_mas_corta(grafo, "P001", "P502")
print(" -> ".join(str(n) for n in ruta))

### Listo!  ahora debrías poder llegar a casa sin problemas